In [82]:
from nltk.corpus import wordnet
sentences= '''"I saw a man with binoculars."
"Flying planes can be dangerous."
"I saw her duck."
"I can't wait to see you!"
"The old man and woman were dancing."
"Police help dog bite victim."
"Scientists find cure for the common cold."
"Kids make nutritious snacks."
"New study shows decline in teen smoking."
"Large hole appears in High Street."
'''
sentences = [s[1: -1] for s in sentences.split('\n') if s.strip()]
sentences

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# 示例句子
sentence = "NLTK is a powerful library for natural language processing."

# 分词
tokens = word_tokenize(sentence)

# 词性标注
tagged = pos_tag(tokens)

# 输出标注结果
print(tagged)


['I saw a man with binoculars.',
 'Flying planes can be dangerous.',
 'I saw her duck.',
 "I can't wait to see you!",
 'The old man and woman were dancing.',
 'Police help dog bite victim.',
 'Scientists find cure for the common cold.',
 'Kids make nutritious snacks.',
 'New study shows decline in teen smoking.',
 'Large hole appears in High Street.']

In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #'last', 'last_expr'

In [ ]:
d = {'text': [[1, 2, 3], [2, 4]], 'label': [0, 1]}
ds = Dataset.from_dict(d)

In [ ]:
dataset_name = "rotten_tomatoes"
dataset = load_dataset(dataset_name, split="train")

In [65]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})

In [66]:
dataset[:3]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'effective but too-tepid biopic'],
 'label': [1, 1, 1]}

In [67]:
for i, example in enumerate(dataset):
    if i == 3: break
    print(example)

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
{'text': 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'label': 1}
{'text': 'effective but too-tepid biopic', 'label': 1}


In [ ]:
cache_dir = '/nas/xd/.cache/torch/transformers/'
model_name = 'EleutherAI/gpt-j-6B'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir=cache_dir)

In [68]:
tokenizer('Good morning!')

{'input_ids': [10248, 3329, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
# def tokenize_function(examples): return tokenizer(example['text'])
tokenized_dataset = dataset.map(lambda example: tokenizer(example['text']),
    batched=True, num_proc=None, remove_columns=dataset.column_names)  # run_clm_flax.py

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns('attention_mask')

In [ ]:
tokenized_dataset

In [69]:
for i, example in enumerate(tokenized_dataset):
    if i == 3: break
    print(example)

{'input_ids': [1169, 3881, 318, 23985, 284, 307, 262, 2310, 301, 4289, 338, 649, 366, 369, 272, 366, 290, 326, 339, 338, 1016, 284, 787, 257, 22870, 772, 3744, 621, 610, 77, 727, 5513, 5767, 89, 44028, 837, 474, 11025, 12, 565, 3885, 5719, 1801, 1326, 393, 2876, 574, 384, 13528, 764]}
{'input_ids': [1169, 17177, 3481, 15962, 24659, 286, 366, 262, 15876, 286, 262, 13917, 366, 26298, 318, 523, 3236, 326, 257, 5721, 286, 2456, 2314, 22668, 6901, 763, 12, 16002, 14, 35248, 279, 2357, 14509, 1559, 338, 9902, 5761, 286, 474, 764, 374, 764, 374, 764, 284, 75, 74, 2013, 338, 3504, 12, 16442, 764]}
{'input_ids': [16803, 475, 1165, 12, 83, 538, 312, 3182, 16603]}


In [ ]:
# mesh-transformer-jax, https://www.tensorflow.org/tutorials/load_data/tfrecord
def _int64_feature(value): return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def write_tfrecords(dataset, fp):
    with tf.io.TFRecordWriter(fp) as writer:
        for example in dataset:
            feature = {"input_ids": _int64_feature(example['input_ids'])}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [ ]:
fp = f'{dataset_name}_train_{len(tokenized_dataset)}.tfrecords'
write_tfrecords(tokenized_dataset, fp)

## load

In [83]:
def _parse_function(example_proto): # https://zhuanlan.zhihu.com/p/552951305
    feature_desc = {"input_ids": tf.io.VarLenFeature(tf.int64)}
    example = tf.io.parse_single_example(example_proto, feature_desc)
#     for name in list(example.keys()):
#         t = example[name]
#         if t.dtype == tf.int64: t = tf.cast(t, dtype=tf.int32)
#         example[name] = tf.sparse.to_dense(t, default_value=0)
        # example[name] = tf.sparse.to_dense(tf.sparse.reorder(t)) # mesh-transformer-jax
    return example

def shard(data, batch_size=None):
    return jax.tree_map(lambda x: x.numpy().reshape(batch_size + x.shape[1:]), data)  # mtj
    
def prefetch(dataset, n_prefetch=None):
    # Taken from: https://github.com/google-research/vision_transformer/blob/master/vit_jax/input_pipeline.py
    ds_iter = iter(dataset)
    ds_iter = map(lambda x: jax.tree_map(lambda t: np.asarray(memoryview(t)), x), ds_iter)
    if n_prefetch: ds_iter = flax.jax_utils.prefetch_to_device(ds_iter, n_prefetch)
    return ds_iter

In [95]:
fp = '/nas/xd/projects/mesh-transformer-jax/rotten_tomatoes_train_8530.tfrecords'
ds = tf.data.TFRecordDataset(fp)
# ds = ds.shuffle(buffer_size=min(1000, len(sequences))) # flaxmodels, https://zhuanlan.zhihu.com/p/552951305
ds = ds.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

In [97]:
for i, example in enumerate(ds):
    if i == 3: break
    print(example)

{'input_ids': <tf.Tensor: shape=(50,), dtype=int32, numpy=
array([ 1169,  3881,   318, 23985,   284,   307,   262,  2310,   301,
        4289,   338,   649,   366,   369,   272,   366,   290,   326,
         339,   338,  1016,   284,   787,   257, 22870,   772,  3744,
         621,   610,    77,   727,  5513,  5767,    89, 44028,   837,
         474, 11025,    12,   565,  3885,  5719,  1801,  1326,   393,
        2876,   574,   384, 13528,   764], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(53,), dtype=int32, numpy=
array([ 1169, 17177,  3481, 15962, 24659,   286,   366,   262, 15876,
         286,   262, 13917,   366, 26298,   318,   523,  3236,   326,
         257,  5721,   286,  2456,  2314, 22668,  6901,   763,    12,
       16002,    14, 35248,   279,  2357, 14509,  1559,   338,  9902,
        5761,   286,   474,   764,   374,   764,   374,   764,   284,
          75,    74,  2013,   338,  3504,    12, 16442,   764],
      dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(9,), d

In [96]:
ds = ds.map(lambda d: {k: tf.cast(tf.sparse.to_dense(v, default_value=0), dtype=tf.int32) for k, v in d.items()})

In [89]:
example['input_ids'].values, example['input_ids'].indices, example['input_ids'].dense_shape

(<tf.Tensor: shape=(22,), dtype=int64, numpy=
 array([  361,   345,  3360,   588,   284,   467,   284,   262,  6918,
          284,   423,  1257,   837,   373, 17914,   318,   257,   922,
         1295,   284,   923,   764])>,
 <tf.Tensor: shape=(22, 1), dtype=int64, numpy=
 array([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21]])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([22])>)

In [72]:
for i, example in enumerate(ds):
    if i == 3: break
    print(example)

{'input_ids': <tf.Tensor: shape=(50,), dtype=int32, numpy=
array([ 1169,  3881,   318, 23985,   284,   307,   262,  2310,   301,
        4289,   338,   649,   366,   369,   272,   366,   290,   326,
         339,   338,  1016,   284,   787,   257, 22870,   772,  3744,
         621,   610,    77,   727,  5513,  5767,    89, 44028,   837,
         474, 11025,    12,   565,  3885,  5719,  1801,  1326,   393,
        2876,   574,   384, 13528,   764], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(53,), dtype=int32, numpy=
array([ 1169, 17177,  3481, 15962, 24659,   286,   366,   262, 15876,
         286,   262, 13917,   366, 26298,   318,   523,  3236,   326,
         257,  5721,   286,  2456,  2314, 22668,  6901,   763,    12,
       16002,    14, 35248,   279,  2357, 14509,  1559,   338,  9902,
        5761,   286,   474,   764,   374,   764,   374,   764,   284,
          75,    74,  2013,   338,  3504,    12, 16442,   764],
      dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(9,), d

In [73]:
gradient_accumulation_steps = 16
# train_mbs_per_replica = 2 # train_micro_batch_size_per_gpu in deepspeed
tpu_size = 8  # jax.num_devices()
cores_per_replica = 4  # mp=4, dp=8/4=2
per_replica_batch = 1  # train_mbs_per_replica in run_clm_mp_xd.py, train_micro_batch_size_per_gpu in deepspeed
train_batch_size = (gradient_accumulation_steps, per_replica_batch * tpu_size // cores_per_replica)
max_len = 80  # max(len(s) for s in sequences) == 78
# ds = ds.apply(tf.data.experimental.dense_to_ragged_batch(np.prod(self.bs), drop_remainder=True)) # mtj
ds = ds.padded_batch(batch_size=np.prod(train_batch_size), padded_shapes={'input_ids': [max_len]},
                     padding_values={'input_ids': 0}, drop_remainder=True)

In [74]:
for batch in ds: print(batch); break

{'input_ids': <tf.Tensor: shape=(32, 80), dtype=int32, numpy=
array([[ 1169,  3881,   318, ...,     0,     0,     0],
       [ 1169, 17177,  3481, ...,     0,     0,     0],
       [16803,   475,  1165, ...,     0,     0,     0],
       ...,
       [  270,  5419,   326, ...,     0,     0,     0],
       [ 5162,  4741,  2308, ...,     0,     0,     0],
       [   64,  4958,   913, ...,     0,     0,     0]], dtype=int32)>}


In [75]:
ds = ds.prefetch(10)  # mesh-transformer-jax
# ds = ds.repeat()  # gpt-neo/inputs.py
# map shard directly over ds won't work, getting AttributeError: 'Tensor' object has no attribute 'numpy'
# because inside tf.function?, see e.g.:
# 1) https://stackoverflow.com/questions/34097281/convert-a-tensor-to-numpy-array-in-tensorflow
# 2) https://github.com/tensorflow/tensorflow/issues/27519
# ds = ds.map(partial(shard, batch_size=train_batch_size), num_parallel_calls=tf.data.AUTOTUNE)
# matthias-wright/flaxmodels/training/stylegan2/data_pipeline.py
ds_iter = iter(ds)
ds_iter = map(lambda x: shard(x, batch_size=train_batch_size), ds_iter)

In [76]:
for batch in ds_iter: print(batch); break

{'input_ids': array([[[ 1169,  3881,   318, ...,     0,     0,     0],
        [ 1169, 17177,  3481, ...,     0,     0,     0]],

       [[16803,   475,  1165, ...,     0,     0,     0],
        [  361,   345,  3360, ...,     0,     0,     0]],

       [[24677,  3212,   355, ...,     0,     0,     0],
        [ 1169,  2646,  3769, ...,     0,     0,     0]],

       ...,

       [[ 2777,  1304,   805, ...,     0,     0,     0],
        [  272,  7306,  2569, ...,     0,     0,     0]],

       [[  265,   546,  6957, ...,     0,     0,     0],
        [  270,  5419,   326, ...,     0,     0,     0]],

       [[ 5162,  4741,  2308, ...,     0,     0,     0],
        [   64,  4958,   913, ...,     0,     0,     0]]], dtype=int32)}


In [77]:
batch['input_ids'].shape

(16, 2, 80)

In [ ]:
@seqio.map_over_dataset
def convert_datatype(ex): return {k: tf.cast(tf.sparse.to_dense(v, default_value=0), dtype=tf.int32) for k, v in ex.items()}

In [ ]:
seqio.TaskRegistry.add(
    "rotten_tomatoes_train_8530",
    seqio.TFExampleDataSource(split_to_filepattern={'train': 'gs://jax_projects/data/rotten_tomatoes_train_8530.tfrecords'},
                             feature_description={"input_ids": tf.io.VarLenFeature(tf.int64)}),
    preprocessors=[convert_datatype,],
)